# Lesson 5: Implementing Pub-Sub for Notifications

## Implementing Pub/Sub for Notifications

Welcome! In this unit, we’ll delve into implementing Pub/Sub for notifications within our Redis-based backend system project. You’ve already learned how to manage user data, handle transactions, and use streams for event logging. Now, we’ll add another powerful feature: real-time notifications using Redis Pub/Sub (publish/subscribe), enabling instant message sending and receiving in our system.

---

### What You’ll Build

In this unit, we’ll focus on creating a simple real-time notification system using Redis Pub/Sub. Specifically, we’ll cover:

- **Publishing Messages**: Sending notifications.
- **Subscribing to Channels**: Receiving and handling notifications.

Here’s a quick refresher on how Pub/Sub works in Redis:

```python
import redis
import json
import time

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

# Function to publish messages to a channel
def publish_message(channel, message):
    client.publish(channel, json.dumps(message))

# Function to handle incoming messages
def message_handler(message):
    data = json.loads(message['data'])
    print(f"Received message from {data['user']}: {data['text']}")

# Function to subscribe to a channel
def subscribe_to_channel(channel):
    pubsub = client.pubsub()
    pubsub.subscribe(**{channel: message_handler})
    return pubsub.run_in_thread(sleep_time=0.001)

# Example usage
channel_name = 'chat_room'
thread = subscribe_to_channel(channel_name)

message = {'user': 'alice', 'text': 'Hello everyone!'}
publish_message('chat_room', message)

# Giving some time for the subscription to set up
time.sleep(1)
thread.stop()
```

### Explanation

In this snippet:
- **`message_handler`** processes incoming messages on the `chat_room` channel.
- **`subscribe_to_channel`** sets up the subscription and runs it in a separate thread.
- **`publish_message`** sends a message to the specified channel, received by `message_handler` and printed to the console.

---

Exciting, isn’t it? Now, it’s time to put this into practice. Let’s implement the complete code to build our real-time notification system.

**Happy coding!**

## Publish Messages to a Channel

Great job in the previous lesson! Now, let's apply what we've learned about Redis Pub/Sub.

In this task, you'll implement the part where messages are published to a Redis channel. This is useful for real-time notifications and messaging systems.

Follow the TODO comments in the starter code to complete the task.

```py
import redis
import json
from datetime import timedelta

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

def add_to_stream(stream_name, entry):
    client.xadd(stream_name, entry)

def read_from_stream(stream_name):
    return client.xread({stream_name: '0-0'})

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Stream name for user activities
stream_name = 'user_activity_stream'

with client.pipeline() as pipeline:
    for user in users:
        user_obj = get_user(user['user_id'])
        if user_obj:
            continue
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
        add_to_stream(stream_name, {
            'event': 'user_added',
            'user_id': user['user_id'],
            'data': json.dumps(user['data']),
            'score': user['score']
        })
    pipeline.execute()

users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})

add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

print("Leaderboard:", get_leaderboard())

for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")

# TODO: Implement the publish_message function that takes a channel name and a message as arguments
    # TODO: Publish message to the channel

# TODO: Create a channel name for notifications called 'chat_room'

# Publish messages to the channel
messages = [
    {'user': 'alice', 'text': 'Hello everyone!'},
    {'user': 'bob', 'text': 'Hi Alice! How are you?'},
    {'user': 'bob', 'text': 'Did you see the latest news?'}
]

# TODO: Publish each message in to the chat room channel



```

## Publish Messages to a Channel

Awesome work in the previous lessons! Now, let’s apply what we’ve learned about Redis Pub/Sub by publishing messages to a Redis channel, a key feature for real-time notifications and messaging systems.

### Instructions

In this task, you’ll implement the function to publish messages to a Redis channel. Follow the `TODO` comments in the code to complete the steps.

```python
import redis
import json
from datetime import timedelta

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

def add_to_stream(stream_name, entry):
    client.xadd(stream_name, entry)

def read_from_stream(stream_name):
    return client.xread({stream_name: '0-0'})

# TODO: Implement the publish_message function that takes a channel name and a message as arguments
def publish_message(channel, message):
    client.publish(channel, json.dumps(message))

# Users and activities
users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Stream and channel names
stream_name = 'user_activity_stream'
channel_name = 'chat_room'  # Chat room channel for notifications

# Adding users to Redis with activities logged
with client.pipeline() as pipeline:
    for user in users:
        user_obj = get_user(user['user_id'])
        if user_obj:
            continue
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
        add_to_stream(stream_name, {
            'event': 'user_added',
            'user_id': user['user_id'],
            'data': json.dumps(user['data']),
            'score': user['score']
        })
    pipeline.execute()

# Update leaderboard
users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})
add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

print("Leaderboard:", get_leaderboard())

for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")

# Messages to publish
messages = [
    {'user': 'alice', 'text': 'Hello everyone!'},
    {'user': 'bob', 'text': 'Hi Alice! How are you?'},
    {'user': 'bob', 'text': 'Did you see the latest news?'}
]

# Publish each message to the 'chat_room' channel
for message in messages:
    publish_message(channel_name, message)
```

### Explanation

- **`publish_message`**: This function takes a `channel` and a `message` argument. It uses Redis’s `publish` function to send the message to the specified channel.
- **Messages in `chat_room`**: After setting up users and the leaderboard, we publish each message from `messages` to the `chat_room` channel.

### Final Notes
In this task, you’ve added real-time notification capabilities, enhancing our Redis system’s interaction with live messaging.

## Handle Incoming Messages with Pub-Sub

Great progress so far! In the last part, you learned how to publish messages to a Redis channel. Now, let's move on to handling incoming messages.

In this task, you'll complete the message_handler function, which processes incoming messages. In addition, you'll implement subscribing to a chat channel using the subscribe_to_channel function.

Fill in the TODO comments to complete the task.

```py
import redis
import json
from datetime import timedelta
import time

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

def add_to_stream(stream_name, entry):
    client.xadd(stream_name, entry)

def read_from_stream(stream_name):
    return client.xread({stream_name: '0-0'})

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Stream name for user activities
stream_name = 'user_activity_stream'

with client.pipeline() as pipeline:
    for user in users:
        user_obj = get_user(user['user_id'])
        if user_obj:
            continue
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
        add_to_stream(stream_name, {
            'event': 'user_added',
            'user_id': user['user_id'],
            'data': json.dumps(user['data']),
            'score': user['score']
        })
    pipeline.execute()

users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})

add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

print("Leaderboard:", get_leaderboard())

# Function to publish messages to a channel
def publish_message(channel, message):
    client.publish(channel, json.dumps(message))

# TODO: Implement the message_handler function that takes 'message' as input
    # TODO: Parse the message data using json.loads

    # TODO: Print the message data with the format "Received message from {user}: {text}"

# TODO: Implement the subscribe_to_channel function that takes 'channel' as input
    # TODO: Create a pubsub instance

    # TODO: Subscribe to the channel with the message_handler function

    # TODO: Run the pubsub instance in a separate thread with a sleep time of 0.001 and return the thread

# Subscribe to a chat channel
channel_name = 'chat_room'

# TODO: Subscribe to the chat channel and store the thread using the subscribe_to_channel function

# Publish messages to the channel
messages = [
    {'user': 'alice', 'text': 'Hello everyone!'},
    {'user': 'bob', 'text': 'Hi Alice! How are you?'},
    {'user': 'bob', 'text': 'Did you see the latest news?'}
]

for msg in messages:
    publish_message(channel_name, msg)

time.sleep(1)

# TODO: Stop the pubsub thread



```

## Handle Incoming Messages with Pub-Sub

Amazing progress so far! Now, let’s tackle handling incoming messages by completing the `message_handler` function to process received messages and the `subscribe_to_channel` function to subscribe to a Redis channel.

### Instructions

- **message_handler**: This function will handle incoming messages by parsing and printing them.
- **subscribe_to_channel**: This function will create a subscription to the specified Redis channel and handle incoming messages in real-time.

```python
import redis
import json
from datetime import timedelta
import time

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

def add_to_stream(stream_name, entry):
    client.xadd(stream_name, entry)

def read_from_stream(stream_name):
    return client.xread({stream_name: '0-0'})

# Users and activities
users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Stream and channel names
stream_name = 'user_activity_stream'
channel_name = 'chat_room'

# Add initial users
with client.pipeline() as pipeline:
    for user in users:
        user_obj = get_user(user['user_id'])
        if user_obj:
            continue
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
        add_to_stream(stream_name, {
            'event': 'user_added',
            'user_id': user['user_id'],
            'data': json.dumps(user['data']),
            'score': user['score']
        })
    pipeline.execute()

# Update leaderboard
users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})
add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

print("Leaderboard:", get_leaderboard())

# Function to publish messages to a channel
def publish_message(channel, message):
    client.publish(channel, json.dumps(message))

# TODO: Implement the message_handler function that takes 'message' as input
def message_handler(message):
    data = json.loads(message['data'])
    print(f"Received message from {data['user']}: {data['text']}")

# TODO: Implement the subscribe_to_channel function that takes 'channel' as input
def subscribe_to_channel(channel):
    pubsub = client.pubsub()
    pubsub.subscribe(**{channel: message_handler})
    return pubsub.run_in_thread(sleep_time=0.001)

# Subscribe to the chat channel and store the thread
thread = subscribe_to_channel(channel_name)

# Messages to publish
messages = [
    {'user': 'alice', 'text': 'Hello everyone!'},
    {'user': 'bob', 'text': 'Hi Alice! How are you?'},
    {'user': 'bob', 'text': 'Did you see the latest news?'}
]

for msg in messages:
    publish_message(channel_name, msg)

time.sleep(1)

# Stop the pubsub thread
thread.stop()
```

### Explanation

- **message_handler**: Handles each incoming message by loading the data and printing it with the sender's username and message.
- **subscribe_to_channel**: Sets up a subscription to the specified channel. The `run_in_thread` method allows the listener to run continuously in a separate thread with a sleep time of 0.001 seconds.
  
### Final Notes

With this setup, you now have a complete Pub/Sub implementation in Redis that handles both publishing and subscribing to channels. Enjoy your real-time notifications system!